# Caderno para construção e teste do BOT

## Loading

In [8]:
## Loader do PDF
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredPDFLoader

path = "C:\\Users\\FHEMIG\Desktop\\Pedro (1)\\Projetos\Zulip Bots\\chatbots-zulip\\data\\manual_impacto.pdf"

## Loaders structured and unstructured

#loader_un = UnstructuredPDFLoader(path)
loader_structured = PyMuPDFLoader(path)

pages_str = loader_structured.load()
#pages_str = loader_un.load()

<>:5: SyntaxWarning: invalid escape sequence '\D'
<>:5: SyntaxWarning: invalid escape sequence '\D'
C:\Users\FHEMIG\AppData\Local\Temp\ipykernel_10884\773998900.py:5: SyntaxWarning: invalid escape sequence '\D'
  path = "C:\\Users\\FHEMIG\Desktop\\Pedro (1)\\Projetos\Zulip Bots\\chatbots-zulip\\data\\manual_impacto.pdf"


## Cleaning

In [9]:
from langchain.text_splitter import CharacterTextSplitter

## Criando texto splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=0
    )

## Split
texts = text_splitter.split_documents(pages_str)




In [11]:
### Testanto

texts[10].page_content

'construído. Nele o usuário SUS é o foco, e o cuidado \nhumanizado, eficiente e de qualidade são a chave para o \nsucesso.  \nPara nos ajudar nessa construção Robert Kaplan e David \nNorton desenvolveram uma ferramenta de gestão chamada \n“Balanced Scorecard (BSC)”. O BSC ajuda a traduzir a visão e a \nestratégia da organização em objetivos claros e mensuráveis, \norganizados em quatro perspectivas: Clientes, que para nós é a \nSociedade, \nAprendizado \ne \nCrescimento, \nProcessos \ne \nFinanceiro: \nClique aqui para \nacessar nosso curso \nsobre Planejamento \nEstratégico, na \nplataforma EAD da \nFhemig.'

In [12]:
## Função para remover espaço em branco

def remove_ws(d):
    text = d.page_content.replace("\n", "")
    d.page_content = text
    return d

# Clean texts by removing whitespace from each document
texts_cleaned = [remove_ws(d) for d in texts]

In [13]:
texts_cleaned[10].page_content



'construído. Nele o usuário SUS é o foco, e o cuidado humanizado, eficiente e de qualidade são a chave para o sucesso.  Para nos ajudar nessa construção Robert Kaplan e David Norton desenvolveram uma ferramenta de gestão chamada “Balanced Scorecard (BSC)”. O BSC ajuda a traduzir a visão e a estratégia da organização em objetivos claros e mensuráveis, organizados em quatro perspectivas: Clientes, que para nós é a Sociedade, Aprendizado e Crescimento, Processos e Financeiro: Clique aqui para acessar nosso curso sobre Planejamento Estratégico, na plataforma EAD da Fhemig.'

In [17]:
## openai

# config.py
import os
from dotenv import load_dotenv

# Carregar as variáveis de ambiente do arquivo .env
load_dotenv(dotenv_path="C:\\Users\\FHEMIG\\Desktop\\Pedro (1)\\Projetos\\Zulip Bots\\chatbots-zulip\\env\\.env")

# Acessar a chave da OpenAI
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Retriever

In [18]:
## retriever openai

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS


# uses OpenAI embeddings to build a retriever
embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
# Creates the document retriever using docs and embeddings
db = FAISS.from_documents(texts_cleaned, embeddings)



# Asking the retriever to do similarity search based on Query
#query = "Foreign Aid for Lowari Road Tunnel & Access Roads Project (2nd Revised )"
#answer = db.similarity_search(query)

# Building the retriever
retriever = db.as_retriever(search_kwargs={'k': 3})


In [ ]:
## Carregando retriever

import joblib

# Carregar o retriever salvo
retriever = joblib.load('data\\retriever.joblib')


## Chain and chats

In [33]:
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Função que o bot usará para responder a mensagens diretas

template = """
Você é **Fabius**, um assistente especializado no Planejamento Estratégico da Fhemig, no Pacto de Gestão Participativa e no sistema InPacto.

### Instruções:

1. Responda de forma objetiva e clara, utilizando as informações disponíveis nos documentos e manuais.
2. Forneça explicações rápidas, sem complicações, e apresente apenas os passos essenciais para a resolução de problemas.
3. Caso necessário, inclua links ou referências a seções específicas dos documentos para mais detalhes.
4. Evite respostas longas. Concentre-se na simplicidade e na eficiência.

### Estrutura de Resposta:
1. **Resumo Rápido**: Explique em uma ou duas frases a solução para a pergunta.
2. **Passos Essenciais** (se aplicável): Liste apenas os passos mínimos necessários para resolver o problema.
3. **Referência ao Manual** (opcional): Inclua uma referência ao manual ou documento, se necessário.

### Exemplos de Perguntas e Respostas:

Exemplo 1:
**Pergunta**: Como faço para cadastrar um novo indicador no sistema InPacto?
**Resposta**:
1. **Resumo Rápido**: Acesse o menu "Indicadores", clique em "Cadastrar Novo Indicador" e preencha os campos obrigatórios.
2. **Passos Essenciais**: 
   - Acesse o InPacto.
   - Vá até "Indicadores" e selecione "Cadastrar Novo Indicador".
   - Preencha os campos e clique em "Salvar".
3. **Referência ao Manual**: Consulte o manual do InPacto, página 34.

Exemplo 2:
**Pergunta**: O que é o Pacto de Gestão Participativa na Fhemig?
**Resposta**:
1. **Resumo Rápido**: O Pacto de Gestão Participativa (PGP) monitora metas e indicadores para alcançar os objetivos estratégicos da Fhemig.
2. **Passos Essenciais**: 
   - A primeira fase é entre a liderança e diretores.
   - A segunda fase envolve diretores e gerentes/setores.
3. **Referência ao Manual**: Ver manual do PGP, página 7.

A sua base de conhecimento, a que você deve se ater é:{context}

A pergunta do usuário é: {query}

"""



# Converter o template em prompt_template

prompt = ChatPromptTemplate.from_template(template)


## Configurar llm com Ollama

llm = ChatOllama(model="llama3.2:1b")

## Configurar chain

chain = chain = (
# The initial dictionary uses the retriever and user supplied query
    {"context":retriever,
     "query":RunnablePassthrough()}
# Feeds that context and query into the prompt then model & lastly 
# uses the ouput parser, do query for the data.
    |  prompt  | llm | StrOutputParser()
)



## Testando!

In [35]:
input = "Como excluir um indicador?"


response = chain.invoke(input)
formatted_response = response.replace("\n", "\n\n")  # Adiciona uma linha em branco após cada nova linha

print(formatted_response)

**Resposta:**

Um indicador pode ser excluído na folha de trabalho "Indicadores". Para fazer isso, vá até o menu "Indicadores" e clique em "Excluir Indicador".



**Passos Essenciais:**



1. Acesse a folha de trabalho "Indicadores".

2. Selecione o indicador que deseja excluir.

3. Clique no ícone de exclusão.



A partir daqui, você também pode editar ou excluir um indicador cadastrado de forma errada.



**Referência ao Manual:** Consulte o manual do InPacto, página 38.
